In [4]:
import numpy as np
from utils import DiscreteDistrib, discrete_wasserstein_distance
from D2_clustering import D2Cluster
from BADMM import badmm_centroid_update
from typing import List
from clustering import clustering

In [5]:
x1 = np.array([[0, 1], [2, 3]])
n1 = x1.shape[0]
x2 =  np.ones((3, 2))
n2 = x2.shape[0]

w1 = np.ones(n1) / n1
w2 = np.ones(n2) / n2

In [6]:
P1 = DiscreteDistrib(w1, x1)
P2 = DiscreteDistrib(w2, x2)
Ps = [P1, P2]

dist, pi = discrete_wasserstein_distance(P1, P2, return_coupling=True)

In [7]:
dist, pi

(1.272019649514065,
 array([[0.16666667, 0.16666667, 0.16666667],
        [0.16666667, 0.16666667, 0.16666667]]))

In [8]:
# c0 = DiscreteDistrib(w=np.array([1/2, 1/2]), x=np.array([[0, 0], [1, 1]]))

In [12]:
c = badmm_centroid_update(Ps, verbose_interval=100, rho=0.01)

Iter: 0, Avg cost 1.151, Primal: 1.2164, Dual: 0.7717
Iter: 100, Avg cost 2.141, Primal: 0.7705, Dual: 0.0001
Iter: 200, Avg cost 2.141, Primal: 0.7715, Dual: 0.0
Iter: 300, Avg cost 2.141, Primal: 0.7719, Dual: 0.0
Iter: 400, Avg cost 2.141, Primal: 0.7721, Dual: 0.0
Iter: 500, Avg cost 2.141, Primal: 0.7722, Dual: 0.0
Iter: 600, Avg cost 2.141, Primal: 0.7723, Dual: 0.0
Iter: 700, Avg cost 2.141, Primal: 0.7723, Dual: 0.0
Iter: 800, Avg cost 2.141, Primal: 0.7724, Dual: 0.0
Iter: 900, Avg cost 2.141, Primal: 0.7724, Dual: 0.0
Early stop activated!


In [9]:
c

[(array([1., 2.]), array([0.00121132])), (array([0.875, 1.375]), array([0.99878868]))]

In [ ]:
clustering(Ps,)